In [275]:
"""
скрипт для разбиения по тепловозниками и электровозникам
"""

'\nскрипт для разбиения по тепловозниками и электровозникам\n'

In [276]:
import pandas as pd
import warnings
import time
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.options.mode.chained_assignment = None  # default='warn'

In [277]:
path_to_machine = 'data/Машинисты.xlsx'
path_to_reit = 'data/Рейтинг машинисты.xlsx'
path_to_person = 'data/Абитуриенты.xlsx'
path_to_end_folder ='data'

In [278]:
machine_df = pd.read_excel(path_to_machine) # распредление по специальностям
reit_df = pd.read_excel(path_to_reit,skiprows=4,header=None) # файл с таблицей из ворда
df_person = pd.read_excel(path_to_person,sheet_name='Абитуриенты',skiprows=8) # данные абитуриентов

In [279]:

# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

In [280]:
# создаем два датафрейма для электровозников и тепловозников
teplo_df = machine_df[['тепловоз','атт.1','мед справка.1']]
teplo_df.dropna(axis=0,inplace=True)
# очищаем от пробельных символов
teplo_df['тепловоз'] = teplo_df['тепловоз'].apply(lambda x:x.strip())
elect_df = machine_df[['электровоз','атт','мед справка']]
elect_df['электровоз'] = elect_df['электровоз'].apply(lambda x:x.strip())

In [281]:
# создаем файл с общим списком
temp_lst = teplo_df['тепловоз'].tolist()
temp_lst.extend(elect_df['электровоз'].tolist())

In [282]:
temp_df = pd.DataFrame(columns=['ФИО общее'])

In [283]:
temp_df['ФИО общее'] = temp_lst

In [284]:
temp_df

,ФИО общее
0,Колоколов Александр Витальевич
1,Воронов Максим Алексеевич
2,Терских Сергей Евгеньевич
3,Машанов Галсан Баторович
4,Чагочкин Кирилл Андреевич
...,...
119,Доржиев Солбон Содбоевич
120,Попов Богдан Сергеевич
121,Воронцов Евгений Михайлович
122,Литвинцев Даниил Денисович


In [285]:
reit_df.head(2)

,0,1,2,3,4,5,6,7,8,9
0,1,154-483-839-91,1,4.85,0,4.847,0,NaN,0,нет
1,2,191-409-787-91,1,4.39,0,4.389,0,NaN,0,да


In [286]:
reit_df.shape

(148, 10)

In [287]:
snils_df = df_person[['ФИО','СНИЛС']]

In [288]:
snils_df = snils_df.drop_duplicates(subset='ФИО') # убираем дубликаты

In [289]:
snils_df.head(2)

,ФИО,СНИЛС
0,Абросова Анастасия Витальевна,147-022-734-38
1,Авдеев Сергей Александрович,155-320-704-36


In [290]:
union_df = snils_df.merge(reit_df,how='outer',left_on='СНИЛС',right_on=1,indicator=True) # объединяем датафреймы

In [291]:
union_df.head(2)

,ФИО,СНИЛС,0,1,2,3,4,5,6,7,8,9,_merge
0,Абросова Анастасия Витальевна,147-022-734-38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Авдеев Сергей Александрович,155-320-704-36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [292]:
union_df[union_df['_merge'] == 'right_only']

,ФИО,СНИЛС,0,1,2,3,4,5,6,7,8,9,_merge
567,NaN,2300564,79.0,2300564,1.0,3.57,0.0,3.571,0.0,NaN,0.0,да,right_only


In [293]:
# сохраняем тех  у кого нет снилс
not_snils = union_df[union_df['_merge'] == 'right_only'] # сохраяем тех у кого нет снилса
not_snils.rename(columns={1:'Личный номер'},inplace=True)
not_snils=not_snils[['Личный номер']]

In [294]:
clean_reit = union_df[union_df['_merge'] =='both'] # отбираем тех кто есть в обоих датафреймах

In [295]:
clean_reit= clean_reit[['ФИО','СНИЛС',2,5]]


In [296]:
clean_reit.head(2)

,ФИО,СНИЛС,2,5
2,Агафонов Семен Владимирович,172-941-745-90,1.0,3.632
4,Айхлер Павел Владимирович,150-435-730-34,1.0,3.476


In [297]:
clean_reit.shape

(147, 4)

In [298]:
# Ищем тех кого нет в файле приемки
missing_priemka = clean_reit.merge(temp_df,how='outer',left_on='ФИО',right_on='ФИО общее',indicator=True)

In [299]:
# файл где содержатся ФИО тех кто есть в рейтинге но кого нет в файле приемки
not_in_priemka =missing_priemka[missing_priemka['_merge'] =='left_only']
not_in_priemka.drop(columns=['ФИО общее','_merge'],inplace=True)
not_in_priemka.rename(columns={2:'Приоритет',5:'Средний балл'})

,ФИО,СНИЛС,Приоритет,Средний балл
6,Бадмаев Александр Артурович,148-583-277-04,1.0,4.000
9,Бастанжиев Кирилл Романович,145-762-360-76,1.0,3.700
10,Батомункуев Саян Баирович,194-729-363-19,2.0,3.550
11,Батуев Виталий Баирович,201-972-374-48,1.0,4.150
14,Болотов Алексей Александрович,153-520-830-39,1.0,3.524
24,Волков Андрей Вадимович,152-278-257-64,1.0,3.714
27,Востриков Федоров Алексеевич,190-381-074-60,1.0,4.286
29,Галданов Бимба Артурович,148-084-046-66,1.0,4.053
30,Галсандоржиев Доржи Батоевич,150-839-818-81,1.0,3.895
32,Гатапов Очир Цыден-Ешиевич,153-103-011-91,1.0,3.750


In [300]:
teplo_df.head(2)

,тепловоз,атт.1,мед справка.1
0,Колоколов Александр Витальевич,да,нет
1,Воронов Максим Алексеевич,да,нет


In [301]:
elect_df.head(2)

,электровоз,атт,мед справка
0,Арьяев Денис Александрович,да,да
1,Данзанов Баян-Мунхэ Жаргалович,да,да


In [302]:
raw_teplo_df =clean_reit.merge(teplo_df,how='outer',left_on='ФИО',right_on='тепловоз',indicator=True)

In [303]:
zabr_df_teplo=raw_teplo_df[raw_teplo_df['_merge'] == 'right_only'] # те кто забрал документы
zabr_df_teplo.drop(columns=['ФИО','СНИЛС',2,5,'_merge'],inplace=True)
zabr_df_teplo

,тепловоз,атт.1,мед справка.1
147,Колодин Иван Александрович,да,нет
148,Раднаев Галсан Вячеславович,да,нет
149,Галсандоржиев Доржи Баторович,да,нет
150,Бальжиев Ринчин Дондокович,да,нет
151,Подшивалов Василий Олегович,да,нет


In [304]:
# электровозы
raw_electo_df =clean_reit.merge(elect_df,how='outer',left_on='ФИО',right_on='электровоз',indicator=True)
zabr_df_electo=raw_electo_df[raw_electo_df['_merge'] == 'right_only'] # те кто забрал документы
zabr_df_electo.drop(columns=['ФИО','СНИЛС',2,5,'_merge'],inplace=True)

In [305]:
clean_teplo_df = raw_teplo_df[raw_teplo_df['_merge']=='both'] # готовим итоговыйй тепловозник
clean_teplo_df.drop(columns=['тепловоз','_merge'],inplace=True)

In [306]:
clean_teplo_df.head(1)

,ФИО,СНИЛС,2,5,атт.1,мед справка.1
0,Агафонов Семен Владимирович,172-941-745-90,1.0,3.632,да,нет


In [307]:
clean_teplo_df.columns = ['ФИО','СНИЛС','Приоритет','Средний балл','Сдан оригинал','Мед.справка']

In [308]:
clean_teplo_df['Приоритет'] = clean_teplo_df['Приоритет'].astype(int)
clean_teplo_df.sort_values(by='Средний балл',ascending=False,inplace=True) # сортируем по убыванию

In [309]:
clean_teplo_df.head()

,ФИО,СНИЛС,Приоритет,Средний балл,Сдан оригинал,Мед.справка
111,Селиванов Максим Дмитриевич,173-248-343-69,1,4.316,да,да
122,Тиунов Вячеслав Романович,149-109-028-57,1,4.167,да,да
124,Толстиков Даниил Константинович,153-684-128-76,1,4.000,да,да
91,Михайлов Агван Станиславович,146-383-727-84,1,3.905,да,да
54,Загузин Александр Павлович,164-465-018-68,1,3.850,да,да


In [310]:
# готовим итоговый электровозник
clean_electo_df = raw_electo_df[raw_electo_df['_merge']=='both'] # готовим итоговыйй тепловозник
clean_electo_df.drop(columns=['электровоз','_merge'],inplace=True)

In [311]:
clean_electo_df.head(1)

,ФИО,СНИЛС,2,5,атт,мед справка
2,Аносов Иван Антонович,151-502-184-16,1.0,3.526,да,да


In [312]:
clean_electo_df.columns = ['ФИО','СНИЛС','Приоритет','Средний балл','Сдан оригинал','Мед.справка']

clean_electo_df['Приоритет'] = clean_electo_df['Приоритет'].astype(int)
clean_electo_df.sort_values(by='Средний балл',ascending=False,inplace=True) # сортируем по убыванию



In [313]:
clean_electo_df.head()

,ФИО,СНИЛС,Приоритет,Средний балл,Сдан оригинал,Мед.справка
101,Попов Богдан Сергеевич,191-409-787-91,1,4.389,да,да
114,Скуратов Егор Викторович,146-115-095-36,1,4.333,да,да
86,Маслов Илья Леонидович,195-583-275-21,1,4.250,нет,нет
77,Лемешев Артем Олегович,146-457-910-88,1,4.158,да,да
76,Латкин Максим Николаевич,165-118-759-74,1,4.150,да,нет


In [314]:
# нужно еще тех кого нет в списке который готовит приемка

In [315]:
with pd.ExcelWriter(f'{path_to_end_folder}/Проверка {current_time}.xlsx') as writer:
    not_snils.to_excel(writer,sheet_name='Нет СНИЛС',index=False)
    not_in_priemka.to_excel(writer,sheet_name='Нет в вашем файле',index=False)
    zabr_df_teplo.to_excel(writer,sheet_name= 'Тепловоз,нет в рейтинге',index=False)
    zabr_df_electo.to_excel(writer,sheet_name= 'Электровоз,нет в рейтинге',index=False)